In [1]:
# To display full output in Notebook, instead of only the last result
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import requests
import json
import time
from bech32 import bech32_encode, convertbits

from datetime import datetime
from tqdm import tqdm
from bech32 import bech32_decode, convertbits
from binascii import hexlify

In [23]:
path = "/home/jovyan/work/New Topic/DeFi.csv"
defi_policy_id_df = pd.read_csv(path)

In [24]:
defi_policy_id_df

,Category,DeFi Name,Token Link,Token Name,Policy ID
0,DEX,Genius Yield,https://cexplorer.io/asset/asset1266q2ewhgul7j...,GENS,dda5fdb1002f7389b33e036b6afee82a8189becb6cba85...
1,DEX,Muesliswap,https://cexplorer.io/asset/asset1pwhywk7x54g73...,MILK,8a1cfae21368b8bebbbed9800fec304e95cce39a2a57dc...
2,DEX,Minswap,https://cexplorer.io/asset/asset1d9v7aptfvpx7w...,MIN,29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a8...
3,DEX,Wingriders,https://cexplorer.io/asset/asset18v68f0vgv2nc3...,WRT,c0ee29a85b13209423b10447d3c2e6a50641a15c57770e...
4,DEX,SundaeSwap,https://cexplorer.io/asset/asset1m4u92ke6820pk...,SUNDAE,9a9693a9a37912a5097918f97918d15240c92ab729a0b7...


In [27]:
def bech32_to_hex(addr_bech32: str) -> str:
    hrp, data = bech32_decode(addr_bech32)
    if data is None:
        raise ValueError(f"Invalid bech32 address: {addr_bech32}")
    decoded = convertbits(data, 5, 8, False)
    return hexlify(bytes(decoded)).decode()

def safe_bech32_to_hex(addr):
    if pd.isna(addr):
        return None
    try:
        return bech32_to_hex(addr)
    except Exception:
        return None

def hex_to_bech32(addr_hex: str, hrp="addr") -> str:
    """Convert 128-char hex string to bech32 address."""
    data = bytes.fromhex(addr_hex)
    five_bit = convertbits(data, 8, 5)
    return bech32_encode(hrp, five_bit)


def safe_hex_to_bech32(addr_hex: str, hrp="addr") -> str:
    """Convert 128-char hex string to bech32 address with error handling."""
    try:
        data = bytes.fromhex(addr_hex)
        five_bit = convertbits(data, 8, 5)
        return bech32_encode(hrp, five_bit)
    except Exception as e:
        return None  

In [28]:
# Your API key
CARDANOSCAN_API_KEY = "520c718b-75dc-4898-aeca-199e059de866"

In [29]:
import requests
import pandas as pd
import time
import json
import re

# Your API key
CARDANOSCAN_API_KEY = "520c718b-75dc-4898-aeca-199e059de866"

# Containers for results
all_assets = {}
failures = {}

# Helper: validate policy ID format
def is_valid_policy_id(pid):
    return isinstance(pid, str) and len(pid) == 56 and re.fullmatch(r'[0-9a-fA-F]{56}', pid)

# Loop through each policy ID
for i, row in defi_policy_id_df.iterrows():
    policy_id = row["Policy ID"]
    defi_name = row["DeFi Name"]

    if not is_valid_policy_id(policy_id):
        print(f"⚠️ Skipping {defi_name}: Invalid policy ID format → {policy_id}")
        failures[defi_name] = "Invalid policy ID format"
        continue

    print(f"🔍 Fetching for {defi_name} ({policy_id[:10]}...)")
    assets = []
    page = 1

    while True:
        url = f"https://api.cardanoscan.io/api/v1/asset/list/byPolicyId?policyId={policy_id}&pageNo={page}"
        headers = {"apiKey": CARDANOSCAN_API_KEY}

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()

            # ✅ FIXED: Use "tokens" field instead of "data"
            page_assets = data.get("tokens", [])
            if not page_assets:
                break

            assets.extend(page_assets)
            page += 1
            time.sleep(1.2)  # Be polite to the API

        except Exception as e:
            print(f"❌ Failed on page {page} for {defi_name}: {e}")
            failures[defi_name] = str(e)
            break

    all_assets[defi_name] = assets
    print(f"✅ {defi_name}: {len(assets)} assets retrieved")

# Save successful results
with open("defi_assets_by_policy.json", "w") as f:
    json.dump(all_assets, f, indent=2)

# Save failure log if any
if failures:
    with open("defi_asset_failures.json", "w") as f:
        json.dump(failures, f, indent=2)
    print(f"⚠️ Some fetches failed. See 'defi_asset_failures.json' for details.")

print("🎉 Done! All data saved to 'defi_assets_by_policy.json'")


🔍 Fetching for Genius Yield (dda5fdb100...)
✅ Genius Yield: 2 assets retrieved
🔍 Fetching for Muesliswap (8a1cfae213...)
✅ Muesliswap: 1 assets retrieved
🔍 Fetching for Minswap (29d222ce76...)
✅ Minswap: 3 assets retrieved
🔍 Fetching for Wingriders (c0ee29a85b...)
✅ Wingriders: 1 assets retrieved
🔍 Fetching for SundaeSwap (9a9693a9a3...)
✅ SundaeSwap: 1 assets retrieved
🎉 Done! All data saved to 'defi_assets_by_policy.json'


In [30]:
!curl --request GET \
  --url "https://api.cardanoscan.io/api/v1/asset/list/byPolicyId?policyId=29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c6&pageNo=1" \
  --header "apiKey: 520c718b-75dc-4898-aeca-199e059de866"


{"tokens":[{"policyId":"29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c6","assetName":"4d494e","fingerprint":"6959ee8569604de7655fe9d2aa59c11da4adf683","assetId":"29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c64d494e","totalSupply":"3000000000000000","txCount":4129320,"mintedOn":"2021-10-10T10:17:49.000Z"},{"policyId":"29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c6","assetName":"4d494e74","fingerprint":"55df7e8b8f955ac65aba1dc984a3c2f95657be7f","assetId":"29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c64d494e74","totalSupply":"45785504224419","txCount":493837,"mintedOn":"2021-12-07T11:03:39.000Z"},{"policyId":"29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c6","assetName":"346434393465","fingerprint":"7b133f04d855b81841b0699c720e6842eea4caa8","assetId":"29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c6346434393465","totalSupply":"0","txCount":3,"mintedOn":"2022-01-10T04:50:08.000Z"}],"count":3,"pageNo":1,"limit":20}

# Get Token User Holders from Blockfrost

In [21]:
import requests
import time
import json
from datetime import datetime

# === CONFIGURATION ===
BLOCKFROST_API_KEY = "mainnetD5tXDiNzInPOA8ABiVQjb9gBJxB2zqXu"
ASSET_CONCAT_ID = "dda5fdb1002f7389b33e036b6afee82a8189becb6cba852e8b79b4fb0014df1047454e53"
OUTPUT_PATH = "genius_yield_gens_holders.json"
MAX_RETRIES = 3
SLEEP_BETWEEN_PAGES = 0.8

# === INIT ===
url_base = f"https://cardano-mainnet.blockfrost.io/api/v0/assets/{ASSET_CONCAT_ID}/addresses"
headers = {"project_id": BLOCKFROST_API_KEY}
holders = []
page = 1
fetching = True

# === PAGINATED FETCH ===
while fetching:
    url = f"{url_base}?page={page}"
    attempt = 1
    success = False

    while attempt <= MAX_RETRIES:
        try:
            response = requests.get(url, headers=headers, timeout=20)

            # Check if response is JSON
            if "application/json" not in response.headers.get("Content-Type", ""):
                print(f"❌ HTML error received instead of JSON on page {page}")
                print(f"📎 Response snippet:\n{response.text[:300]}")
                raise ValueError("Non-JSON response received")

            if response.status_code == 200:
                page_data = response.json()

                if not page_data:
                    print(f"🚫 Page {page} is empty. Stopping fetch.")
                    fetching = False  # ✅ Stop the outer loop
                    success = True
                    break

                holders.extend(page_data)
                print(f"✅ Page {page}: {len(page_data)} addresses fetched")
                success = True
                break
            else:
                print(f"⚠️ Attempt {attempt}: {response.status_code} - {response.text}")

        except Exception as e:
            print(f"⚠️ Attempt {attempt}: Exception - {e}")

        attempt += 1
        time.sleep(3)

    if not success:
        print(f"❌ Failed to fetch page {page}. Stopping.")
        break

    page += 1
    try:
        time.sleep(SLEEP_BETWEEN_PAGES)
    except KeyboardInterrupt:
        print("🛑 Manually interrupted during sleep. Exiting fetch loop.")
        break

# === SAVE TO FILE ===
if holders:
    with open(OUTPUT_PATH, "w") as f:
        json.dump(holders, f, indent=2)
    print(f"🎉 Done! Total holders: {len(holders)}. Saved to '{OUTPUT_PATH}'")
else:
    print("⚠️ No data saved. Holder list is empty.")

print(f"✅ Script finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


✅ Page 1: 100 addresses fetched
✅ Page 2: 100 addresses fetched
✅ Page 3: 100 addresses fetched
✅ Page 4: 100 addresses fetched
✅ Page 5: 100 addresses fetched
✅ Page 6: 100 addresses fetched
✅ Page 7: 100 addresses fetched
✅ Page 8: 100 addresses fetched
✅ Page 9: 100 addresses fetched
✅ Page 10: 100 addresses fetched
✅ Page 11: 100 addresses fetched
✅ Page 12: 100 addresses fetched
✅ Page 13: 100 addresses fetched
✅ Page 14: 100 addresses fetched
✅ Page 15: 100 addresses fetched
✅ Page 16: 100 addresses fetched
✅ Page 17: 100 addresses fetched
✅ Page 18: 100 addresses fetched
✅ Page 19: 100 addresses fetched
✅ Page 20: 100 addresses fetched
✅ Page 21: 100 addresses fetched
✅ Page 22: 100 addresses fetched
✅ Page 23: 100 addresses fetched
✅ Page 24: 100 addresses fetched
✅ Page 25: 100 addresses fetched
✅ Page 26: 100 addresses fetched
✅ Page 27: 100 addresses fetched
✅ Page 28: 100 addresses fetched
✅ Page 29: 100 addresses fetched
✅ Page 30: 100 addresses fetched
✅ Page 31: 100 addr

In [33]:
import requests
import time
import json
from datetime import datetime

# === CONFIGURATION ===
BLOCKFROST_API_KEY = "mainnetD5tXDiNzInPOA8ABiVQjb9gBJxB2zqXu"
ASSET_CONCAT_ID = "8a1cfae21368b8bebbbed9800fec304e95cce39a2a57dc35e2e3ebaa4d494c4b"  # MILK
OUTPUT_PATH = "muesliswap_milk_holders.json"

MAX_RETRIES = 3
SLEEP_BETWEEN_PAGES = 0.8  # safe speed

# === INIT ===
url_base = f"https://cardano-mainnet.blockfrost.io/api/v0/assets/{ASSET_CONCAT_ID}/addresses"
headers = {"project_id": BLOCKFROST_API_KEY}
holders = []
page = 1
fetching = True

# === PAGINATED FETCH LOOP ===
while fetching:
    url = f"{url_base}?page={page}"
    attempt = 1
    success = False

    while attempt <= MAX_RETRIES:
        try:
            response = requests.get(url, headers=headers, timeout=20)

            # Check response is JSON
            if "application/json" not in response.headers.get("Content-Type", ""):
                print(f"❌ HTML error instead of JSON on page {page}")
                print(f"📎 Snippet:\n{response.text[:300]}")
                raise ValueError("Non-JSON response")

            if response.status_code == 200:
                page_data = response.json()

                if not page_data:
                    print(f"🚫 Page {page} is empty. Stopping.")
                    fetching = False
                    success = True
                    break

                holders.extend(page_data)
                print(f"✅ Page {page}: {len(page_data)} addresses fetched")
                success = True
                break
            else:
                print(f"⚠️ Attempt {attempt}: Status {response.status_code} - {response.text}")

        except Exception as e:
            print(f"⚠️ Attempt {attempt}: Exception - {e}")

        attempt += 1
        time.sleep(3)

    if not success:
        print(f"❌ Failed to fetch page {page}. Stopping.")
        break

    page += 1
    try:
        time.sleep(SLEEP_BETWEEN_PAGES)
    except KeyboardInterrupt:
        print("🛑 Interrupted. Exiting.")
        break

# === SAVE RESULTS ===
if holders:
    with open(OUTPUT_PATH, "w") as f:
        json.dump(holders, f, indent=2)
    print(f"🎉 Done! Total holders: {len(holders)}. Saved to '{OUTPUT_PATH}'")
else:
    print("⚠️ No data saved. Empty result.")

print(f"✅ Script finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


✅ Page 1: 100 addresses fetched
✅ Page 2: 100 addresses fetched
✅ Page 3: 100 addresses fetched
✅ Page 4: 100 addresses fetched
✅ Page 5: 100 addresses fetched
✅ Page 6: 100 addresses fetched
✅ Page 7: 100 addresses fetched
✅ Page 8: 100 addresses fetched
✅ Page 9: 100 addresses fetched
✅ Page 10: 100 addresses fetched
✅ Page 11: 100 addresses fetched
✅ Page 12: 100 addresses fetched
✅ Page 13: 100 addresses fetched
✅ Page 14: 100 addresses fetched
✅ Page 15: 100 addresses fetched
✅ Page 16: 100 addresses fetched
✅ Page 17: 100 addresses fetched
✅ Page 18: 100 addresses fetched
✅ Page 19: 100 addresses fetched
✅ Page 20: 100 addresses fetched
✅ Page 21: 100 addresses fetched
✅ Page 22: 100 addresses fetched
✅ Page 23: 100 addresses fetched
✅ Page 24: 100 addresses fetched
✅ Page 25: 100 addresses fetched
✅ Page 26: 100 addresses fetched
✅ Page 27: 100 addresses fetched
✅ Page 28: 100 addresses fetched
✅ Page 29: 100 addresses fetched
✅ Page 30: 100 addresses fetched
✅ Page 31: 100 addr

In [34]:
import requests
import time
import json
from datetime import datetime

# === CONFIGURATION ===
BLOCKFROST_API_KEY = "mainnetD5tXDiNzInPOA8ABiVQjb9gBJxB2zqXu"
ASSET_CONCAT_ID = "29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c64d494e"
OUTPUT_PATH = "minswap_min_holders.json"
MAX_RETRIES = 5
SLEEP_BETWEEN_PAGES = 1.2  
TIMEOUT_SECONDS = 60

# === INIT ===
url_base = f"https://cardano-mainnet.blockfrost.io/api/v0/assets/{ASSET_CONCAT_ID}/addresses"
headers = {"project_id": BLOCKFROST_API_KEY}
holders = []
page = 1
fetching = True

# === PAGINATED FETCH ===
while fetching:
    url = f"{url_base}?page={page}"
    attempt = 1
    success = False

    while attempt <= MAX_RETRIES:
        try:
            response = requests.get(url, headers=headers, timeout=TIMEOUT_SECONDS)

            # Check if response is JSON
            if "application/json" not in response.headers.get("Content-Type", ""):
                print(f"❌ HTML error received instead of JSON on page {page}")
                print(f"📎 Response snippet:\n{response.text[:300]}")
                raise ValueError("Non-JSON response received")

            if response.status_code == 200:
                page_data = response.json()

                if not page_data:
                    print(f"🚫 Page {page} is empty. Stopping fetch.")
                    fetching = False
                    success = True
                    break

                holders.extend(page_data)
                print(f"✅ Page {page}: {len(page_data)} addresses fetched")
                success = True
                break
            else:
                print(f"⚠️ Attempt {attempt} on page {page}: {response.status_code} - {response.text}")

        except Exception as e:
            print(f"⚠️ Attempt {attempt} on page {page}: Exception - {e}")

        attempt += 1
        time.sleep(5)

    if not success:
        print(f"❌ Failed to fetch page {page}. Stopping.")
        break

    page += 1
    try:
        time.sleep(SLEEP_BETWEEN_PAGES)
    except KeyboardInterrupt:
        print("🛑 Manually interrupted during sleep. Exiting fetch loop.")
        break

# === SAVE TO FILE ===
if holders:
    with open(OUTPUT_PATH, "w") as f:
        json.dump(holders, f, indent=2)
    print(f"🎉 Done! Total holders: {len(holders)}. Saved to '{OUTPUT_PATH}'")
else:
    print("⚠️ No data saved. Holder list is empty.")

print(f"✅ Script finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


❌ HTML error received instead of JSON on page 1
📎 Response snippet:
<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!
⚠️ Attempt 1 on page 1: Exception - Non-JSON response received
❌ HTML error received instead of JSON on page 1
📎 Response snippet:
<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!
⚠️ Attempt 2 on page 1: Exception - Non-JSON response received
✅ Page 1: 100 addresses fetched
✅ Page 2: 100 addresses fetched
❌ HTML error received instead of JSON on page 3
📎 Response snippet:
<!DO

In [35]:
import requests
import time
import json
from datetime import datetime

# === CONFIGURATION ===
BLOCKFROST_API_KEY = "mainnetD5tXDiNzInPOA8ABiVQjb9gBJxB2zqXu"
ASSET_CONCAT_ID = "c0ee29a85b13209423b10447d3c2e6a50641a15c57770e27cb9d507357696e67526964657273"
OUTPUT_PATH = "wingriders_wrt_holders.json"
MAX_RETRIES = 5
SLEEP_BETWEEN_PAGES = 1.2
TIMEOUT_SECONDS = 60

# === INIT ===
url_base = f"https://cardano-mainnet.blockfrost.io/api/v0/assets/{ASSET_CONCAT_ID}/addresses"
headers = {"project_id": BLOCKFROST_API_KEY}
holders = []
page = 1
fetching = True

# === PAGINATED FETCH ===
while fetching:
    url = f"{url_base}?page={page}"
    attempt = 1
    success = False

    while attempt <= MAX_RETRIES:
        try:
            response = requests.get(url, headers=headers, timeout=TIMEOUT_SECONDS)

            if "application/json" not in response.headers.get("Content-Type", ""):
                print(f"❌ HTML error received instead of JSON on page {page}")
                print(f"📎 Response snippet:\n{response.text[:300]}")
                raise ValueError("Non-JSON response received")

            if response.status_code == 200:
                page_data = response.json()

                if not page_data:
                    print(f"🚫 Page {page} is empty. Stopping fetch.")
                    fetching = False
                    success = True
                    break

                holders.extend(page_data)
                print(f"✅ Page {page}: {len(page_data)} addresses fetched")
                success = True
                break
            else:
                print(f"⚠️ Attempt {attempt} on page {page}: {response.status_code} - {response.text}")

        except Exception as e:
            print(f"⚠️ Attempt {attempt} on page {page}: Exception - {e}")

        attempt += 1
        time.sleep(5)

    if not success:
        print(f"❌ Failed to fetch page {page}. Stopping.")
        break

    page += 1
    try:
        time.sleep(SLEEP_BETWEEN_PAGES)
    except KeyboardInterrupt:
        print("🛑 Manually interrupted during sleep. Exiting fetch loop.")
        break

# === SAVE TO FILE ===
if holders:
    with open(OUTPUT_PATH, "w") as f:
        json.dump(holders, f, indent=2)
    print(f"🎉 Done! Total holders: {len(holders)}. Saved to '{OUTPUT_PATH}'")
else:
    print("⚠️ No data saved. Holder list is empty.")

print(f"✅ Script finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


❌ HTML error received instead of JSON on page 1
📎 Response snippet:
<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!
⚠️ Attempt 1 on page 1: Exception - Non-JSON response received
❌ HTML error received instead of JSON on page 1
📎 Response snippet:
<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!
⚠️ Attempt 2 on page 1: Exception - Non-JSON response received
✅ Page 1: 100 addresses fetched
✅ Page 2: 100 addresses fetched
✅ Page 3: 100 addresses fetched
✅ Page 4: 100 addresses fetched
✅ Page 5

In [36]:
import requests
import time
import json
from datetime import datetime

# === CONFIGURATION ===
BLOCKFROST_API_KEY = "mainnetD5tXDiNzInPOA8ABiVQjb9gBJxB2zqXu"
ASSET_CONCAT_ID = "9a9693a9a37912a5097918f97918d15240c92ab729a0b7c4aa144d7753554e444145"
OUTPUT_PATH = "sundaeswap_SUNDAE_holders.json"
MAX_RETRIES = 5
SLEEP_BETWEEN_PAGES = 1.2
TIMEOUT_SECONDS = 60

# === INIT ===
url_base = f"https://cardano-mainnet.blockfrost.io/api/v0/assets/{ASSET_CONCAT_ID}/addresses"
headers = {"project_id": BLOCKFROST_API_KEY}
holders = []
page = 1
fetching = True

# === PAGINATED FETCH ===
while fetching:
    url = f"{url_base}?page={page}"
    attempt = 1
    success = False

    while attempt <= MAX_RETRIES:
        try:
            response = requests.get(url, headers=headers, timeout=TIMEOUT_SECONDS)

            if "application/json" not in response.headers.get("Content-Type", ""):
                print(f"❌ HTML error received instead of JSON on page {page}")
                print(f"📎 Response snippet:\n{response.text[:300]}")
                raise ValueError("Non-JSON response received")

            if response.status_code == 200:
                page_data = response.json()

                if not page_data:
                    print(f"🚫 Page {page} is empty. Stopping fetch.")
                    fetching = False
                    success = True
                    break

                holders.extend(page_data)
                print(f"✅ Page {page}: {len(page_data)} addresses fetched")
                success = True
                break
            else:
                print(f"⚠️ Attempt {attempt} on page {page}: {response.status_code} - {response.text}")

        except Exception as e:
            print(f"⚠️ Attempt {attempt} on page {page}: Exception - {e}")

        attempt += 1
        time.sleep(5)

    if not success:
        print(f"❌ Failed to fetch page {page}. Stopping.")
        break

    page += 1
    try:
        time.sleep(SLEEP_BETWEEN_PAGES)
    except KeyboardInterrupt:
        print("🛑 Manually interrupted during sleep. Exiting fetch loop.")
        break

# === SAVE TO FILE ===
if holders:
    with open(OUTPUT_PATH, "w") as f:
        json.dump(holders, f, indent=2)
    print(f"🎉 Done! Total holders: {len(holders)}. Saved to '{OUTPUT_PATH}'")
else:
    print("⚠️ No data saved. Holder list is empty.")

print(f"✅ Script finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


❌ HTML error received instead of JSON on page 1
📎 Response snippet:
<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!
⚠️ Attempt 1 on page 1: Exception - Non-JSON response received
✅ Page 1: 100 addresses fetched
✅ Page 2: 100 addresses fetched
✅ Page 3: 100 addresses fetched
❌ HTML error received instead of JSON on page 4
📎 Response snippet:
<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!
⚠️ Attempt 1 on page 4: Exception - Non-JSON response received
✅ Page 4: 100 addresses fetched
✅ Page 5

# Make them to be Dataframe

In [38]:
# === Load JSON file ===
with open("wingriders_wrt_holders.json", "r") as f:
    data = json.load(f)

# === Parse into DataFrame ===
WRT_holders_df = pd.DataFrame([{
    "token": "WRT",
    "bech32_address": entry["address"],
    "quantity": int(entry["quantity"])
} for entry in data])

# === Optional: sort by quantity descending ===
WRT_holders_df = WRT_holders_df.sort_values(by="quantity", ascending=False).reset_index(drop=True)

# === Show preview ===
print(WRT_holders_df.head())

  token                                     bech32_address        quantity
0   WRT  addr1wypr0np3xatwhddulsnj3aaac65qg768zgs2xpd2x...  18802768670327
1   WRT  addr1zxhew7fmsup08qvhdnkg8ccra88pw7q5trrncja3d...  15889017227655
2   WRT  addr1wy9z0v8mrkhtyll43fu6mnhu0p87tna48xt4p5649...  13797083749206
3   WRT  addr1q8fhnpzgdukm4fc8xatkpx6kmdev7h22ze8nm52l6...   8805566666666
4   WRT  addr1qyr6ddcy3cc0srahvqxkzxctr4h84nml9ngtsdlue...   7560817668505


In [39]:
# === Load JSON file ===
with open("sundaeswap_sundae_holders.json", "r") as f:
    data = json.load(f)

# === Parse into DataFrame ===
SUNDAE_holders_df = pd.DataFrame([{
    "token": "SUNDAE",
    "bech32_address": entry["address"],
    "quantity": int(entry["quantity"])
} for entry in data])

# === Optional: sort by quantity descending ===
SUNDAE_holders_df = SUNDAE_holders_df.sort_values(by="quantity", ascending=False).reset_index(drop=True)

# === Show preview ===
print(SUNDAE_holders_df.head())

    token                                     bech32_address         quantity
0  SUNDAE  addr1w9742z4fewans7ry6cjp95pc4ecv7y54cx298lp5q...  660000000000000
1  SUNDAE  addr1z8srqftqemf0mjlukfszd97ljuxdp44r372txfcr7...  244714351879802
2  SUNDAE  addr1q95vk0ufx0g76xrgt8h74uvtha386735en3sng38d...  108601423359914
3  SUNDAE  addr1wyv9f6gz32y5jm5hw2j53qnjn5t92nuwmxhj0mrqg...   90101210271085
4  SUNDAE  addr1qyg45ed0w6eur2k283nju4usc4vt5sntxrrn47ela...   76890387542655


In [40]:
# === Load JSON file ===
with open("muesliswap_milk_holders.json", "r") as f:
    data = json.load(f)

# === Parse into DataFrame ===
MILK_holders_df = pd.DataFrame([{
    "token": "MILK",
    "bech32_address": entry["address"],
    "quantity": int(entry["quantity"])
} for entry in data])

# === Optional: sort by quantity descending ===
MILK_holders_df = MILK_holders_df.sort_values(by="quantity", ascending=False).reset_index(drop=True)

# === Show preview ===
print(MILK_holders_df.head())

  token                                     bech32_address  quantity
0  MILK  addr1q9rtclgcvqwhutjnkr3acfgetxn2f6qjkzjcdc4m5...   7089683
1  MILK  addr1qxfskza5zy0l5j2u4hwsegvrmv446a5x6954nq6rq...   1329203
2  MILK  addr1qygtklglhrru2y4hja2h09d3ejtvy2ekxnyk9w47c...    271945
3  MILK  addr1zyq0kyrml023kwjk8zr86d5gaxrt5w8lxnah8r6m6...    166203
4  MILK  addr1qx4z4rhewajfp76elhcl4v832a6c3splvzg6xgx5z...    114892


In [41]:
# === Load JSON file ===
with open("minswap_min_holders.json", "r") as f:
    data = json.load(f)

# === Parse into DataFrame ===
MIN_holders_df = pd.DataFrame([{
    "token": "MIN",
    "bech32_address": entry["address"],
    "quantity": int(entry["quantity"])
} for entry in data])

# === Optional: sort by quantity descending ===
MIN_holders_df = MIN_holders_df.sort_values(by="quantity", ascending=False).reset_index(drop=True)

# === Show preview ===
print(MIN_holders_df.head())

  token                                     bech32_address         quantity
0   MIN  addr1zx0wxal6dz7rjzxk2mwfvj9564rp9uajqrscftx44...  998422624088847
1   MIN  addr1w9jk84m6fk5p4yvrr35dry4ramhmqdq0zdq4rz4w0...  428049739523097
2   MIN  addr1z84q0denmyep98ph3tmzwsmw0j7zau9ljmsqx6a4r...  234793515755785
3   MIN  addr1z87vw6ts32hywu4j4kyk9qfgd36zhzx3y7fc786vg...  229125022327324
4   MIN  addr1wy3fscaws62d59k6qqhg3xsarx7vstzczgjmdhx2j...  207620627540886


In [42]:
# === Load JSON file ===
with open("genius_yield_gens_holders.json", "r") as f:
    data = json.load(f)

# === Parse into DataFrame ===
GENS_holders_df = pd.DataFrame([{
    "token": "GENS",
    "bech32_address": entry["address"],
    "quantity": int(entry["quantity"])
} for entry in data])

# === Optional: sort by quantity descending ===
GENS_holders_df = GENS_holders_df.sort_values(by="quantity", ascending=False).reset_index(drop=True)

# === Show preview ===
print(GENS_holders_df.head())

  token                                     bech32_address        quantity
0  GENS  addr1w8r99sv75y9tqfdzkzyqdqhedgnef47w4x7y0qnyt...  49239973887509
1  GENS  addr1qxn4kd8jfrmaz2atgevhdjpa3960yycec84jcg5ns...   2188888557244
2  GENS  addr1q9wcps6z0dynwhqh0f7mmvfnrf48c47dms208gtjn...   1931545644200
3  GENS  addr1z84q0denmyep98ph3tmzwsmw0j7zau9ljmsqx6a4r...   1903410702243
4  GENS  addr1vy38wnftcwrugm2umfyzr76rj9vqfp6rsc5vak22f...   1550805270786


In [44]:

# === Blockfrost Configuration ===
API_KEY = "mainnetD5tXDiNzInPOA8ABiVQjb9gBJxB2zqXu"
headers = {"project_id": API_KEY}

# === Asset ID List ===
asset_ids = [
    # GENS
    "dda5fdb1002f7389b33e036b6afee82a8189becb6cba852e8b79b4fb0014df1047454e53",
    # MILK
    "8a1cfae21368b8bebbbed9800fec304e95cce39a2a57dc35e2e3ebaa4d494c4b",
    # MIN
    "29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c64d494e",
    # WRT
    "c0ee29a85b13209423b10447d3c2e6a50641a15c57770e27cb9d507357696e67526964657273",
    # SUNDAE
    "9a9693a9a37912a5097918f97918d15240c92ab729a0b7c4aa144d7753554e444145"
]

# === Query and Print Decimals ===
for aid in asset_ids:
    try:
        response = requests.get(
            f"https://cardano-mainnet.blockfrost.io/api/v0/assets/{aid}",
            headers=headers,
            timeout=15
        )
        response.raise_for_status()
        metadata = response.json().get("metadata", {})
        decimals = metadata.get("decimals", None)
        print(f"{aid} → decimals = {decimals}")
    except Exception as e:
        print(f"{aid} → ❌ Error: {e}")


dda5fdb1002f7389b33e036b6afee82a8189becb6cba852e8b79b4fb0014df1047454e53 → decimals = 6
8a1cfae21368b8bebbbed9800fec304e95cce39a2a57dc35e2e3ebaa4d494c4b → decimals = 0
29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c64d494e → decimals = 6
c0ee29a85b13209423b10447d3c2e6a50641a15c57770e27cb9d507357696e67526964657273 → decimals = 6
9a9693a9a37912a5097918f97918d15240c92ab729a0b7c4aa144d7753554e444145 → decimals = 6


# Deal with Quantity Decimals and Define Holders are those hold with at Least One Token Amount

In [76]:
# === Token Decimals Mapping ===
token_decimals = {
    "WRT": 6,
    "SUNDAE": 6,
    "MILK": 0,
    "MIN": 6,
    "GENS": 6,
}

def add_token_amount(df, token_name):
    decimals = token_decimals.get(token_name, 0)
    df["token_amount"] = df["quantity"].astype(float) / (10 ** decimals)
    df = df[df["token_amount"] >= 1].reset_index(drop=True)  #only left holders who hold at least 1 token.
    return df

# === Apply to all your existing token dfs ===
WRT_holders_df = add_token_amount(WRT_holders_df, "WRT")
SUNDAE_holders_df = add_token_amount(SUNDAE_holders_df, "SUNDAE")
MILK_holders_df = add_token_amount(MILK_holders_df, "MILK")
MIN_holders_df = add_token_amount(MIN_holders_df, "MIN")
GENS_holders_df = add_token_amount(GENS_holders_df, "GENS")


In [55]:
pd.options.display.float_format = "{:.6f}".format


In [67]:
SUNDAE_holders_df.head()

,token,bech32_address,quantity,token_amount
0,SUNDAE,addr1w9742z4fewans7ry6cjp95pc4ecv7y54cx298lp5q...,660000000000000,660000000.000000
1,SUNDAE,addr1z8srqftqemf0mjlukfszd97ljuxdp44r372txfcr7...,244714351879802,244714351.879802
2,SUNDAE,addr1q95vk0ufx0g76xrgt8h74uvtha386735en3sng38d...,108601423359914,108601423.359914
3,SUNDAE,addr1wyv9f6gz32y5jm5hw2j53qnjn5t92nuwmxhj0mrqg...,90101210271085,90101210.271085
4,SUNDAE,addr1qyg45ed0w6eur2k283nju4usc4vt5sntxrrn47ela...,76890387542655,76890387.542655


In [74]:
GENS_holders_df['bech32_address'][16986]

'addr1q9tak4pwag00uzhtsd8gsfw5ya5vr374mhj0zxzanw7ar3dcvtjm8enawhyjjkcf6eves2cwz4c8y9tvhjuzpvmu4rwsum2dzf'

In [81]:
MILK_holders_df.tail()

,token,bech32_address,quantity,token_amount
4181,MILK,addr1q8r3hzgmv0fga9fzmm30ak5ecqf4yz8ch6c8n7vz3...,1,1.000000
4182,MILK,addr1q8mhpak6hcf0jt5yycrkval5sdde4plgycvvynznd...,1,1.000000
4183,MILK,addr1q8t7y4z7uj7rz9sxgjpc3sck7lyur42l6j4jghgys...,1,1.000000
4184,MILK,addr1qyzzjguu93at6f4eygyhuz627xqv0musna20jw66r...,1,1.000000
4185,MILK,addr1qynhphtxrdzwx32h494qllflfqrh5d6jc8u4ddhfg...,1,1.000000


In [83]:
# === Compute both raw and unique holder counts for each token ===
token_holder_stats = []

for token, df in {
    "WRT": WRT_holders_df,
    "SUNDAE": SUNDAE_holders_df,
    "MILK": MILK_holders_df,
    "MIN": MIN_holders_df,
    "GENS": GENS_holders_df,
}.items():
    total_rows = len(df)
    unique_holders = df["bech32_address"].nunique()
    token_holder_stats.append({
        "Token": token,
        "Address Records": total_rows,
        "Unique Holders": unique_holders
    })

# === Display as DataFrame ===
token_holder_stats_df = pd.DataFrame(token_holder_stats)
print(token_holder_stats_df)


    Token  Address Records  Unique Holders
0     WRT             9978            9978
1  SUNDAE            81661           81661
2    MILK             4186            4186
3     MIN            25840           25840
4    GENS            13652           13652


# Change Bech32 Address to Hex Address

In [84]:
# === Add hex_address column to each df ===
WRT_holders_df["hex_address"]    = WRT_holders_df["bech32_address"].apply(safe_bech32_to_hex)
SUNDAE_holders_df["hex_address"] = SUNDAE_holders_df["bech32_address"].apply(safe_bech32_to_hex)
MILK_holders_df["hex_address"]   = MILK_holders_df["bech32_address"].apply(safe_bech32_to_hex)
MIN_holders_df["hex_address"]    = MIN_holders_df["bech32_address"].apply(safe_bech32_to_hex)
GENS_holders_df["hex_address"]   = GENS_holders_df["bech32_address"].apply(safe_bech32_to_hex)


# Save Token Holder Dataframes

In [86]:
WRT_holders_df.to_csv("WRT_holders_df.csv", index=False)

In [87]:
SUNDAE_holders_df.to_csv("SUNDAE_holders_df.csv", index=False)

In [88]:
MILK_holders_df.to_csv("MILK_holders_df.csv", index=False)

In [89]:
MIN_holders_df.to_csv("MIN_holders_df.csv", index=False)

In [90]:
GENS_holders_df.to_csv("GENS_holders_df.csv", index=False)